In [3]:
rootDirectory = '/home/ec2-user/FrequencyBounds/'
import sys
sys.path.append(rootDirectory + 'Source')
import pandas as pd
import pickle
from SQLParser import *

In [24]:
def removeFKJoins(queryJG):
    newJG = JoinQueryGraph()
    for vertex in queryJG.vertexDict.values():
        newJG.addAlias(vertex.tableName, vertex.alias)
        
    for vertex in queryJG.vertexDict.values():
        for pred in vertex.predicates:
            newJG.addPredicate(vertex.alias, pred.colName, pred.predType, pred.compValue)
        for i in range(len(vertex.outputJoinCols)):
            if vertex.outputJoinCols[i] == "MOVIE_ID" and vertex.edgeJoinCols[i] == "MOVIE_ID":  
                newJG.addAlias("TITLE", "t")  
                newJG.addJoin('t', 'id', vertex.alias, 'MOVIE_ID')
                newJG.addJoin('t', 'id', vertex.edgeAliases[i], 'MOVIE_ID')
            else: 
                newJG.addJoin(vertex.alias, vertex.outputJoinCols[i], vertex.edgeAliases[i], vertex.edgeJoinCols[i])
    newJG.buildJoinGraph(addImpliedJoins=False)
    return newJG



def getNeuroCardCSVLine(queryJG, trueCard):
    line = ""
    prefix = ""
    for alias, table in queryJG.tableDict.items():
        line += prefix
        line += table + " " + alias
        prefix = ","
    line += "#"
    prefix = ""
    joinStrings = []
    for leftTable, edges in queryJG.edgeDict.items():
        for rightTable, rightColumn, leftColumn in edges:
            if leftTable <= rightTable:
                line += prefix
                line += leftTable +"." + leftColumn +"="+rightTable+"."+rightColumn
                prefix = ","
    
    predConverter = {"NOT LIKE" : "NOT_LIKE",
                     "IS NULL" : "IS_NULL",
                     "IS NOT NULL": "IS_NOT_NULL",
                    "NOT IN" : "NOT_IN"}
    
    line = line.lower()
    line += "#"
    prefix = ""
    for alias, vertex in queryJG.vertexDict.items():
        for predicate in vertex.predicates:
            line += prefix
            predicateType = predicate.predType.upper()
            if predicateType in predConverter:
                predicateType = predConverter[predicateType]
            line += alias.lower() + "." + predicate.colName.lower() + "," + predicateType + "," 
            compValue = None
            if isinstance(predicate.compValue, str):
                compValue = "'" + predicate.compValue + "'"
                if "," in compValue:
                    compValue = "\"" + compValue + "\""
            else:
                compValue = predicate.compValue
            
            if predicate.predType == "IN":
                listPrefix=""
                line += "\"("
                for value in compValue:
                    if isinstance(value, str):
                        line += listPrefix + "'" + value + "'"
                    else:
                        line += listPrefix + value 
                    listPrefix = ","
                line += ")\""
            else:
                line += str(compValue)
            prefix = ","        
    line += "#" + str(int(trueCard)) +"\n"
    return line
        

In [25]:
benchmark = "JOBM"
queryJGs = None        
dbConn = None
if benchmark == 'JOBLight':
    queryJGs =  SQLFileToJoinQueryGraphs(rootDirectory + 'Workloads/JOBLightQueries.sql')
elif benchmark == 'JOBLightRanges':
    queryJGs =  SQLFileToJoinQueryGraphs(rootDirectory + 'Workloads/JOBLightRangesQueries.sql')
elif benchmark == 'JOBM':
    queryJGs =  SQLFileToJoinQueryGraphs(rootDirectory + 'Workloads/JOBMQueries.sql')

allQueryHints = pickle.load(open("/home/ec2-user/FrequencyBounds/StatObjects/TrueCardinality_" + benchmark + ".pkl", 'rb'))
queryLabels = []
subQueryTables = []
neuroCardCSVLines = []
hintCSVLines = []
for i, queryJG in enumerate(queryJGs):
    queryHints = allQueryHints[i]
    for j, subQuery in enumerate(queryJG.getIntermediateQueries()):
        tables = list(subQuery.vertexDict.keys())
        hint = [x for x in queryHints if set(x.tables) == set(tables)][0]
        trueRows = hint.rowEstimate
        subQueryNoFK = removeFKJoins(subQuery)
        neuroCardCSVLines.append(getNeuroCardCSVLine(subQueryNoFK, trueRows))
        queryLabels.append(i)
        subQueryTables.append(tables)
        hintCSVLines.append(str(i) + "|" + ",".join(tables) + "\n")
neuroCardFile = open("/home/ec2-user/FrequencyBounds/Workloads/NeuroCard_" + benchmark + ".csv", "w")
neuroCardFile.writelines(neuroCardCSVLines)

hintCSVFile = open("/home/ec2-user/FrequencyBounds/Workloads/NeuroCardHintIndex_" + benchmark + ".csv", "w")
hintCSVFile.writelines(hintCSVLines)

In [75]:
queryJGs[0].vertexDict["MC"].outputJoinCols

['MOVIE_ID', 'MOVIE_ID']

In [136]:
len(hintCSVLines)

9472